<a href="https://colab.research.google.com/github/santiagonajera/PLANEACION-Y-PROGRAMACION-DE-LA-PRODUCCION/blob/main/abc_xyz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

# Cargar el archivo Excel
url = 'https://github.com/santiagonajera/PLANEACION-Y-PROGRAMACION-DE-LA-PRODUCCION/raw/refs/heads/main/ventas-2026-forecast-UPS.xlsx'

# Leer las dos hojas
df_forecast = pd.read_excel(url, sheet_name='Forecast')
df_precios = pd.read_excel(url, sheet_name='Precios-Costos')

print("Datos de Forecast:")
print(df_forecast.head())
print("\nDatos de Precios-Costos:")
print(df_precios.head())

# Calcular el total de ventas por SKU (suma de todos los meses)
# Suponiendo que las columnas de meses son desde la segunda columna en adelante
columnas_meses = df_forecast.columns[1:]  # Excluir la columna SKU
df_forecast['Total_Ventas'] = df_forecast[columnas_meses].sum(axis=1)

# Hacer merge con los precios
df_analisis = df_forecast[['SKU', 'Total_Ventas']].merge(
    df_precios[['SKU', 'Precio']],
    on='SKU',
    how='left'
)

# Calcular el valor total de ventas (cantidad * precio)
df_analisis['Valor_Total'] = df_analisis['Total_Ventas'] * df_analisis['Precio']

# Ordenar por valor total descendente
df_analisis = df_analisis.sort_values('Valor_Total', ascending=False).reset_index(drop=True)

# Calcular el porcentaje acumulado
df_analisis['Porcentaje'] = (df_analisis['Valor_Total'] / df_analisis['Valor_Total'].sum()) * 100
df_analisis['Porcentaje_Acumulado'] = df_analisis['Porcentaje'].cumsum()

# Clasificación ABC
def clasificar_abc(porcentaje_acum):
    if porcentaje_acum <= 60:
        return 'A'
    elif porcentaje_acum <= 80:
        return 'B'
    else:
        return 'C'

df_analisis['Clasificacion_ABC'] = df_analisis['Porcentaje_Acumulado'].apply(clasificar_abc)

# Mostrar resultados
print("\n" + "="*80)
print("CLASIFICACIÓN ABC")
print("="*80)
print(df_analisis.to_string(index=False))

# Resumen por clasificación
print("\n" + "="*80)
print("RESUMEN POR CLASIFICACIÓN")
print("="*80)
resumen = df_analisis.groupby('Clasificacion_ABC').agg({
    'SKU': 'count',
    'Valor_Total': 'sum',
    'Porcentaje': 'sum'
}).rename(columns={'SKU': 'Cantidad_SKUs'})

print(resumen)

# Guardar resultados en Excel
output_file = 'clasificacion_abc_resultados.xlsx'
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    df_analisis.to_excel(writer, sheet_name='Clasificacion_ABC', index=False)
    resumen.to_excel(writer, sheet_name='Resumen')

print(f"\n✓ Resultados guardados en: {output_file}")

Datos de Forecast:
     SKU  2026-01-01 00:00:00  2026-02-01 00:00:00  2026-03-01 00:00:00  \
0  SKU 1                24258                26532                27222   
1  SKU 2                42808                48743                53289   
2  SKU 3                32407                37713                38826   
3  SKU 4                37750                40805                43475   
4  SKU 5                22099                26808                28054   

   2026-04-01 00:00:00  2026-05-01 00:00:00  2026-06-01 00:00:00  \
0                24418                25849                31533   
1                47401                53264                63001   
2                35447                38890                45952   
3                41505                44500                52594   
4                25545                26621                31864   

   2026-07-01 00:00:00  2026-08-01 00:00:00  2026-09-01 00:00:00  \
0                33135                35926          

In [2]:
import pandas as pd
import numpy as np
from scipy.stats.mstats import winsorize

# Cargar el archivo Excel
url = 'https://github.com/santiagonajera/PLANEACION-Y-PROGRAMACION-DE-LA-PRODUCCION/raw/refs/heads/main/ventas-2026-forecast-UPS.xlsx'

# Leer la hoja de histórico
df_historico = pd.read_excel(url, sheet_name='Historico')

print("Datos Históricos:")
print(df_historico.head())
print(f"\nShape: {df_historico.shape}")

# Obtener las columnas de meses (todas excepto SKU)
columnas_meses = df_historico.columns[1:]
print(f"\nMeses disponibles: {len(columnas_meses)}")
print(f"Columnas: {list(columnas_meses)}")

# Tomar solo los últimos 22 meses
if len(columnas_meses) > 22:
    columnas_22meses = columnas_meses[-22:]
    print(f"\nUsando los últimos 22 meses: desde {columnas_22meses[0]} hasta {columnas_22meses[-1]}")
else:
    columnas_22meses = columnas_meses
    print(f"\nUsando todos los {len(columnas_22meses)} meses disponibles")

# Crear DataFrame para análisis
df_analisis = df_historico[['SKU']].copy()

# Función para aplicar winsorización y calcular coeficiente de variación
def calcular_cv_winsorizado(serie):
    """
    Aplica winsorización y calcula el coeficiente de variación
    """
    # Convertir a array y eliminar NaN
    datos = serie.dropna().values

    if len(datos) < 2:
        return np.nan

    # Aplicar winsorización (limitar outliers al 5% en cada extremo)
    datos_winsor = winsorize(datos, limits=[0.05, 0.05])

    # Calcular media y desviación estándar
    media = np.mean(datos_winsor)
    std = np.std(datos_winsor, ddof=1)

    # Calcular coeficiente de variación
    if media != 0:
        cv = (std / abs(media)) * 100
    else:
        cv = np.nan

    return cv

# Calcular CV para cada SKU
print("\nCalculando coeficientes de variación con winsorización...")
cv_list = []

for idx, row in df_historico.iterrows():
    sku = row['SKU']
    serie = row[columnas_22meses]
    cv = calcular_cv_winsorizado(serie)
    cv_list.append({
        'SKU': sku,
        'CV': cv,
        'Media': serie.mean(),
        'Desv_Std': serie.std()
    })

df_analisis = pd.DataFrame(cv_list)

# Eliminar SKUs con CV no válido
df_analisis = df_analisis.dropna(subset=['CV']).reset_index(drop=True)

# Calcular percentiles
percentil_33 = df_analisis['CV'].quantile(0.33)
percentil_67 = df_analisis['CV'].quantile(0.67)

print(f"\nPercentil 33: {percentil_33:.2f}%")
print(f"Percentil 67: {percentil_67:.2f}%")

# Clasificación XYZ
def clasificar_xyz(cv):
    """
    X: CV bajo (hasta percentil 33) - Demanda estable
    Y: CV medio (percentil 33 a 67) - Demanda moderadamente variable
    Z: CV alto (mayor a percentil 67) - Demanda muy variable
    """
    if cv <= percentil_33:
        return 'X'
    elif cv <= percentil_67:
        return 'Y'
    else:
        return 'Z'

df_analisis['Clasificacion_XYZ'] = df_analisis['CV'].apply(clasificar_xyz)

# Ordenar por CV
df_analisis = df_analisis.sort_values('CV').reset_index(drop=True)

# Mostrar resultados
print("\n" + "="*80)
print("CLASIFICACIÓN XYZ POR COEFICIENTE DE VARIACIÓN")
print("="*80)
print(df_analisis.to_string(index=False))

# Resumen por clasificación
print("\n" + "="*80)
print("RESUMEN POR CLASIFICACIÓN XYZ")
print("="*80)
resumen_xyz = df_analisis.groupby('Clasificacion_XYZ').agg({
    'SKU': 'count',
    'CV': ['min', 'max', 'mean']
}).round(2)
resumen_xyz.columns = ['Cantidad_SKUs', 'CV_Min', 'CV_Max', 'CV_Promedio']

print(resumen_xyz)

# Interpretación
print("\n" + "="*80)
print("INTERPRETACIÓN")
print("="*80)
print("Clase X: Demanda ESTABLE (bajo coeficiente de variación)")
print("         → Menor incertidumbre, fácil de pronosticar")
print("\nClase Y: Demanda MODERADAMENTE VARIABLE")
print("         → Incertidumbre media, requiere seguimiento")
print("\nClase Z: Demanda MUY VARIABLE (alto coeficiente de variación)")
print("         → Alta incertidumbre, difícil de pronosticar")

# Guardar resultados en Excel
output_file = 'clasificacion_xyz_resultados.xlsx'
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    df_analisis.to_excel(writer, sheet_name='Clasificacion_XYZ', index=False)
    resumen_xyz.to_excel(writer, sheet_name='Resumen_XYZ')

print(f"\n✓ Resultados guardados en: {output_file}")

# Mostrar estadísticas adicionales
print("\n" + "="*80)
print("ESTADÍSTICAS DE COEFICIENTES DE VARIACIÓN")
print("="*80)
print(f"CV Mínimo: {df_analisis['CV'].min():.2f}%")
print(f"CV Máximo: {df_analisis['CV'].max():.2f}%")
print(f"CV Promedio: {df_analisis['CV'].mean():.2f}%")
print(f"CV Mediana: {df_analisis['CV'].median():.2f}%")

Datos Históricos:
     SKU  2022-01-01 00:00:00  2022-02-01 00:00:00  2022-03-01 00:00:00  \
0  SKU 1                19965                21423                23381   
1  SKU 2                34309                40503                42970   
2  SKU 3                26988                31236                33004   
3  SKU 4                30843                35385                37745   
4  SKU 5                19121                22141                23779   

   2022-04-01 00:00:00  2022-05-01 00:00:00  2022-06-01 00:00:00  \
0                21679                22308                25813   
1                39138                42344                48322   
2                30251                32731                37715   
3                34830                37973                42389   
4                21004                23353                26432   

   2022-07-01 00:00:00  2022-08-01 00:00:00  2022-09-01 00:00:00  ...  \
0                24818                30160      

In [3]:
import pandas as pd
import numpy as np
from scipy.stats.mstats import winsorize
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("ANÁLISIS ABC-XYZ DE INVENTARIOS")
print("="*80)

# Cargar el archivo Excel
url = 'https://github.com/santiagonajera/PLANEACION-Y-PROGRAMACION-DE-LA-PRODUCCION/raw/refs/heads/main/ventas-2026-forecast-UPS.xlsx'

# ==================== PARTE 1: CLASIFICACIÓN ABC ====================
print("\n[1/3] Calculando clasificación ABC (basada en valor de ventas)...")

# Leer datos de forecast y precios
df_forecast = pd.read_excel(url, sheet_name='Forecast')
df_precios = pd.read_excel(url, sheet_name='Precios-Costos')

# Calcular total de ventas proyectadas 2026
columnas_meses_forecast = df_forecast.columns[1:]
df_forecast['Total_Ventas'] = df_forecast[columnas_meses_forecast].sum(axis=1)

# Merge con precios
df_abc = df_forecast[['SKU', 'Total_Ventas']].merge(
    df_precios[['SKU', 'Precio']],
    on='SKU',
    how='left'
)

# Calcular valor total
df_abc['Valor_Total'] = df_abc['Total_Ventas'] * df_abc['Precio']

# Ordenar y calcular porcentaje acumulado
df_abc = df_abc.sort_values('Valor_Total', ascending=False).reset_index(drop=True)
df_abc['Porcentaje'] = (df_abc['Valor_Total'] / df_abc['Valor_Total'].sum()) * 100
df_abc['Porcentaje_Acumulado'] = df_abc['Porcentaje'].cumsum()

# Clasificación ABC
def clasificar_abc(porcentaje_acum):
    if porcentaje_acum <= 60:
        return 'A'
    elif porcentaje_acum <= 80:
        return 'B'
    else:
        return 'C'

df_abc['Clasificacion_ABC'] = df_abc['Porcentaje_Acumulado'].apply(clasificar_abc)

print("✓ Clasificación ABC completada")

# ==================== PARTE 2: CLASIFICACIÓN XYZ ====================
print("\n[2/3] Calculando clasificación XYZ (basada en coeficiente de variación)...")

# Leer datos históricos
df_historico = pd.read_excel(url, sheet_name='Historico')

# Obtener últimos 22 meses
columnas_meses_hist = df_historico.columns[1:]
if len(columnas_meses_hist) > 22:
    columnas_22meses = columnas_meses_hist[-22:]
else:
    columnas_22meses = columnas_meses_hist

# Función para calcular CV con winsorización
def calcular_cv_winsorizado(serie):
    datos = serie.dropna().values
    if len(datos) < 2:
        return np.nan

    # Winsorización al 5%
    datos_winsor = winsorize(datos, limits=[0.05, 0.05])
    media = np.mean(datos_winsor)
    std = np.std(datos_winsor, ddof=1)

    if media != 0:
        cv = (std / abs(media)) * 100
    else:
        cv = np.nan

    return cv

# Calcular CV para cada SKU
cv_list = []
for idx, row in df_historico.iterrows():
    sku = row['SKU']
    serie = row[columnas_22meses]
    cv = calcular_cv_winsorizado(serie)
    cv_list.append({'SKU': sku, 'CV': cv})

df_xyz = pd.DataFrame(cv_list)
df_xyz = df_xyz.dropna(subset=['CV']).reset_index(drop=True)

# Calcular percentiles
percentil_33 = df_xyz['CV'].quantile(0.33)
percentil_67 = df_xyz['CV'].quantile(0.67)

# Clasificación XYZ
def clasificar_xyz(cv):
    if cv <= percentil_33:
        return 'X'
    elif cv <= percentil_67:
        return 'Y'
    else:
        return 'Z'

df_xyz['Clasificacion_XYZ'] = df_xyz['CV'].apply(clasificar_xyz)

print("✓ Clasificación XYZ completada")

# ==================== PARTE 3: COMBINACIÓN ABC-XYZ ====================
print("\n[3/3] Generando matriz ABC-XYZ combinada...")

# Merge de ambas clasificaciones
df_final = df_abc.merge(df_xyz[['SKU', 'CV', 'Clasificacion_XYZ']], on='SKU', how='left')

# Crear clasificación combinada
df_final['Clasificacion_ABC-XYZ'] = df_final['Clasificacion_ABC'] + df_final['Clasificacion_XYZ']

# Ordenar por importancia (ABC primero, luego XYZ)
orden_abc = {'A': 1, 'B': 2, 'C': 3}
orden_xyz = {'X': 1, 'Y': 2, 'Z': 3}
df_final['Orden_ABC'] = df_final['Clasificacion_ABC'].map(orden_abc)
df_final['Orden_XYZ'] = df_final['Clasificacion_XYZ'].map(orden_xyz)
df_final = df_final.sort_values(['Orden_ABC', 'Orden_XYZ']).reset_index(drop=True)

# Eliminar columnas auxiliares
df_final = df_final.drop(['Orden_ABC', 'Orden_XYZ'], axis=1)

# Redondear valores
df_final['Valor_Total'] = df_final['Valor_Total'].round(2)
df_final['Porcentaje'] = df_final['Porcentaje'].round(2)
df_final['Porcentaje_Acumulado'] = df_final['Porcentaje_Acumulado'].round(2)
df_final['CV'] = df_final['CV'].round(2)

print("✓ Matriz ABC-XYZ completada")

# ==================== RESULTADOS ====================
print("\n" + "="*80)
print("CLASIFICACIÓN ABC-XYZ COMPLETA")
print("="*80)
print(df_final[['SKU', 'Total_Ventas', 'Precio', 'Valor_Total', 'Porcentaje_Acumulado',
               'Clasificacion_ABC', 'CV', 'Clasificacion_XYZ', 'Clasificacion_ABC-XYZ']].to_string(index=False))

# Resumen por clasificación ABC
print("\n" + "="*80)
print("RESUMEN CLASIFICACIÓN ABC")
print("="*80)
resumen_abc = df_final.groupby('Clasificacion_ABC').agg({
    'SKU': 'count',
    'Valor_Total': 'sum',
    'Porcentaje': 'sum'
}).rename(columns={'SKU': 'Cantidad_SKUs', 'Valor_Total': 'Valor_Total_$'})
print(resumen_abc.round(2))

# Resumen por clasificación XYZ
print("\n" + "="*80)
print("RESUMEN CLASIFICACIÓN XYZ")
print("="*80)
resumen_xyz = df_final.groupby('Clasificacion_XYZ').agg({
    'SKU': 'count',
    'CV': ['min', 'max', 'mean']
}).round(2)
resumen_xyz.columns = ['Cantidad_SKUs', 'CV_Min', 'CV_Max', 'CV_Promedio']
print(resumen_xyz)

# Matriz ABC-XYZ
print("\n" + "="*80)
print("MATRIZ ABC-XYZ (Cantidad de SKUs)")
print("="*80)
matriz = pd.crosstab(df_final['Clasificacion_ABC'], df_final['Clasificacion_XYZ'])
print(matriz)

# Matriz con porcentaje
print("\n" + "="*80)
print("MATRIZ ABC-XYZ (Porcentaje de SKUs)")
print("="*80)
matriz_pct = pd.crosstab(df_final['Clasificacion_ABC'], df_final['Clasificacion_XYZ'], normalize='all') * 100
print(matriz_pct.round(1))

# Interpretación por categoría
print("\n" + "="*80)
print("INTERPRETACIÓN DE CATEGORÍAS ABC-XYZ")
print("="*80)
interpretaciones = {
    'AX': 'Alta importancia, demanda estable → IDEAL (máxima prioridad)',
    'AY': 'Alta importancia, demanda moderada → Monitoreo constante',
    'AZ': 'Alta importancia, demanda variable → Inventario de seguridad alto',
    'BX': 'Importancia media, demanda estable → Gestión eficiente',
    'BY': 'Importancia media, demanda moderada → Revisión periódica',
    'BZ': 'Importancia media, demanda variable → Flexibilidad en pedidos',
    'CX': 'Baja importancia, demanda estable → Pedidos programados',
    'CY': 'Baja importancia, demanda moderada → Gestión básica',
    'CZ': 'Baja importancia, demanda variable → Política de mínimos'
}

for categoria, descripcion in interpretaciones.items():
    count = len(df_final[df_final['Clasificacion_ABC-XYZ'] == categoria])
    if count > 0:
        print(f"{categoria}: {descripcion} ({count} SKUs)")

# Recomendaciones estratégicas
print("\n" + "="*80)
print("RECOMENDACIONES ESTRATÉGICAS")
print("="*80)
print("Clase A (60% del valor):")
print("  • Monitoreo diario de inventarios")
print("  • Pronósticos precisos y frecuentes")
print("  • Relaciones estrechas con proveedores")
print("\nClase B (20% del valor):")
print("  • Monitoreo semanal")
print("  • Pronósticos mensuales")
print("  • Gestión estándar de pedidos")
print("\nClase C (20% del valor):")
print("  • Monitoreo mensual")
print("  • Pedidos según necesidad")
print("  • Simplificar gestión")
print("\nClase X (baja variabilidad):")
print("  • Inventarios tipo pull (bajo pedido)")
print("  • Stock de seguridad bajo")
print("\nClase Y (variabilidad media):")
print("  • Balance entre push y pull")
print("  • Stock de seguridad moderado")
print("\nClase Z (alta variabilidad):")
print("  • Sistema push con colchón")
print("  • Stock de seguridad alto")

# Guardar resultados
output_file = 'clasificacion_abc_xyz_completa.xlsx'
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    df_final.to_excel(writer, sheet_name='Clasificacion_Completa', index=False)
    resumen_abc.to_excel(writer, sheet_name='Resumen_ABC')
    resumen_xyz.to_excel(writer, sheet_name='Resumen_XYZ')
    matriz.to_excel(writer, sheet_name='Matriz_ABC-XYZ')

    # Hoja de interpretación
    df_interpretacion = pd.DataFrame(list(interpretaciones.items()),
                                     columns=['Categoria', 'Interpretacion'])
    df_interpretacion.to_excel(writer, sheet_name='Interpretacion', index=False)

print(f"\n✓ Resultados guardados en: {output_file}")
print("\n" + "="*80)
print("ANÁLISIS COMPLETADO")
print("="*80)

ANÁLISIS ABC-XYZ DE INVENTARIOS

[1/3] Calculando clasificación ABC (basada en valor de ventas)...
✓ Clasificación ABC completada

[2/3] Calculando clasificación XYZ (basada en coeficiente de variación)...
✓ Clasificación XYZ completada

[3/3] Generando matriz ABC-XYZ combinada...
✓ Matriz ABC-XYZ completada

CLASIFICACIÓN ABC-XYZ COMPLETA
   SKU  Total_Ventas  Precio  Valor_Total  Porcentaje_Acumulado Clasificacion_ABC    CV Clasificacion_XYZ Clasificacion_ABC-XYZ
 SKU 2        637915    5.84   3725423.60                  6.76                 A 15.64                 X                    AX
 SKU 3        486842    6.26   3047630.92                 18.25                 A 14.90                 X                    AX
 SKU 4        549669    5.41   2973709.29                 23.65                 A 15.25                 X                    AX
SKU 11        500089    5.29   2645470.81                 33.29                 A 15.88                 X                    AX
SKU 17        3140